<img src="logoucm.png" style="height: 100px">

<h1><center>Calidad del aire en la Comunidad de Madrid</center></h1>

# Resumen

El Índice de Calidad del Aire (CAQI) es un valor adimensional que se calcula a partir de los datos recopilados en las estaciones de monitoreo, teniendo en cuenta los límites establecidos por la legislación y los efectos nocivos para la salud de varios contaminantes.

El objetivo de este trabajo de fin de grado es desarrollar índices de calidad del aire pronosticados para cada una de las cinco zonas del municipio de Madrid. Además, se presentarán diferentes métodos para realizar una predicción diaria de la calidad del aire en la Comunidad de Madrid, utilizando la información disponible en el siguiente <a href="https://airedemadrid.madrid.es/portales/calidadaire/es/Servicios-y-recomendaciones/Prediccion-diaria/?vgnextfmt=default&vgnextchannel=e6ae471c5c503710VgnVCM1000008a4a900aRCRD">enlace</a>. 

# Introducción

Los datos utilizados en este trabajo fueron recopilados diariamente desde 2010 hasta 2023 por la Comunidad de Madrid. Estos datos se refieren a la concentración de diversos contaminantes medidos en 24 estaciones distribuidas en todo el territorio de la comunidad. La Comunidad de Madrid publica estos datos diariamente en su página web, en el Portal de Calidad del Aire, junto con un Índice de Calidad del Aire. El objetivo de este índice es proporcionar una evaluación de los efectos de la contaminación atmosférica en la salud.

Para obtener más información sobre las variables recopiladas, se puede consultar el Interprete fichero en el siguiente enlace: <a href="https://airedemadrid.madrid.es/portal/site/calidadaire">Interprete fichero</a>.

A continuación, se presenta una breve descripción de los datos recopilados:

| Atributo | Descripción |
| :- |:- |
|**PROVINCIA**| Identificador de la provincia (en este caso siempre '28'-Madrid)|
|**MUNICIPIO**| Identificador del municipio (en este caso siempre '79'-Mardrid|
|**ESTACION**| Identificador de la estación de recogida de los contaminantes|
|**MAGNITUD**| Identificador del contaminante|
|**PUNTO MUESTREO**| Identificador del punto de muestro|
|**ANO**| Año de recogida del dato|
|**MES**| Mes de recogida el dato|
|**D01**| D0_X_ indica el día del mes|
|**V01**| V0_X_ indica si el dato está verificado o no (Sí: "V", No: "N")|

# 0. Librerías

In [ ]:
# Librerias 
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt  

%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# 1. Análisis exploratorio de los datos

## 1.1 Limpieza de la base de datos

En primer lugar, procedemos a abrir el fichero que contiene los datos y analizamos su dimensión y estructura.

In [ ]:
# Importacion de datos
df = pd.read_csv("./Datos/datos_total.csv", delimiter = ",")
df_original = df.copy()

df.head(5)

In [ ]:
# Eliminacion de variables
df = df.drop(columns=['Unnamed: 0', 'PROVINCIA', 'MUNICIPIO'], axis = 1)

# Dimension de los datos
df.shape

Disponemos de un total de 21.633 registros que contienen información sobre 70 variables. Sin embargo, hemos eliminado las variables **_PROVINCIA_** y **_MUNICIPIO_** debido a su redundancia, ya que los datos se refieren exclusivamente a la Comunidad de Madrid. Además, hemos eliminado la variable **_Unnamed:0_**. Por lo tanto, la base de datos ahora se compone de las 67 variables restantes para cada uno de los 21.633 registros.

In [ ]:
# Tipo de las variables y valores nulos
df.info()

A primera vista, la base de datos no muestra ningún valor faltante. Sin embargo, es importante tener en cuenta que existen datos no verificados. Esto significa que, aunque no haya valores explícitamente marcados como faltantes, algunos datos pueden no haber sido validados o requerir una verificación adicional para asegurar su precisión. En este contexto, sólo son válidos aquellos datos que estén acompañados por un código de verificación "V". 

Para abordar este problema, se reducirá la dimensión de la base de datos, conservando únicamente los compuestos presentes en la creación del índice de calidad del aire. Esto permitirá enfocar el análisis en las variables más relevantes para evaluar la calidad del aire. Esto es:

*Los compuestos que se emplean para calcular el índice de calidad son las partículas en suspensión (PM10 y PM2,5), dióxido de azufre, dióxido de nitrógeno y ozono. Para cada uno de estos contaminantes se establece un índice parcial, de forma que el peor valor de los cinco definirá el índice global y, por lo tanto, la calidad del aire en el municipio de Madrid.*

| Contaminantes | id| Muy bueno | Bueno | Regular | Malo | Muy malo |
| :- |:- |:-|:-|:-|:-| :-|
|Partículas PM2,5| 9| 0-15|16-30|31-55|56-110|>110|
|Partículas PM10| 10| 0-25|26-50|51-90|91-180|>180|
|NO2| 8| 0-50|51-100|101-200|201-400|>400|
|O3| 14|0-60|61-120|121-180|181-240|>240|
|SO2| 1|0-50|51-100|101-350|351-500|>500|
[Indices parciales](#indices-parciales)

Fuente: [Aire de Madrid](https://airedemadrid.madrid.es/portales/calidadaire/es/Bases-de-datos-y-publicaciones/Bases-de-datos-de-calidad-del-aire/Indices-y-zonas/Indice-de-Calidad-del-Aire/?vgnextfmt=default&vgnextoid=303d635a41187710VgnVCM1000001d4a900aRCRD&vgnextchannel=480285a1259d7710VgnVCM2000001f4a900aRCRD)

En consecuencia, se procede a la creación de una nueva base de datos con una estructura más apropiada para el análisis.

In [ ]:
# Dataframe con los contaminantes de interes
df_contaminantes = df[(df['MAGNITUD'].isin([9, 10, 8, 14, 1]))]

# Listado de dias y correspondientes codigos de verificacion
dias = ['D01','D02','D03','D04','D05','D06','D07','D08','D09','D10',
       'D11','D12','D13','D14','D15','D16','D17','D18','D19','D20',
       'D21','D22','D23','D24','D25','D26','D27','D28','D29','D30','D31']

verf = ['V01','V02','V03','V04','V05','V06','V07','V08','V09','V10',
       'V11','V12','V13','V14','V15','V16','V17','V18','V19','V20',
       'V21','V22','V23','V24','V25','V26','V27','V28','V29','V30','V31']

# Formato long del dataframe
# Dias
df_dias = df_contaminantes.melt(id_vars=['ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO', 'ANO', 'MES'],
                                value_vars = dias,
                                var_name = 'DIA',
                                value_name = 'VALOR')
# Verificaciones
df_verf= df_contaminantes.melt(id_vars=['ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO', 'ANO', 'MES'],
                               value_vars = verf,
                               var_name = 'VER',
                               value_name = 'VV')

# Asociar los valores de verificacion
df_dias['VV'] = df_verf['VV']
df = df_dias

# Visualizacion
df.head(5)

Con este dataframe, vamos a diferenciar entre los valores no verificados (aquellos que están seguidos de una código de verificación 'N') y los valores no medidos debido a la inexistencia de un día en particular (por ejemplo, el 30 de febrero).

Para comenzar, eliminamos del dataframe aquellos registros correspondientes a los días que no existen.

In [ ]:
# Creacion de la columna 'date'
date_str = df['ANO'].astype(str) + '-' + df['MES'].astype(str) + '-' + df['DIA'].str.replace('D', '').astype(str)
df['date'] = pd.to_datetime(date_str, format='%Y-%m-%d', errors='coerce')

# Eliminacion de los registros con dias inexistentes
df = df.dropna(subset=['date'])

# Reordanacion de las columnas
df = df[['ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO', 'ANO', 'MES', 'DIA', 'date', 'VALOR', 'VV']]

# Reordenacion de los registros por fecha
df = df.sort_values(['ESTACION','MAGNITUD','ANO','MES'])

En segundo lugar, vamos a asignar el valor $-1$ a los datos no verificados para poder llevar a cabo una imputación de dichos valores. De esta manera, en análisis posteriores, podremos diferenciar los datos no verificados de aquellos que no están registrados.

In [ ]:
# Asignacion de valores missing
df.loc[df['VV'] != 'V', 'VALOR'] = -1
df = df.drop(['VV'], axis = 1)

Por último, estudiamos la presencia de valores nulos y datos no verificados y nos aseguramos de que no existan filas duplicadas.

In [ ]:
# Comprobacion valores nulos y duplicados
print("Numero de valores nulos en las diferentes variables:") 
print(df.isnull().sum())
print("\nNumero de datos no verificados:", (df['VALOR'] == -1).sum())
print("\nNumero de registros duplicados:" ,df.duplicated().sum())

## 1.2 Imputación de missing

Una primera aproximación para tratar los datos missing podría ser utilizar la técnica de interpolación lineal. La interpolación lineal calcula un valor promedio entre dos puntos conocidos y este se utiliza para reemplazar el valor faltante. Es importante destacar que, al utilizar la interpolación lineal, estamos asumiendo una relación lineal entre los datos.

Otra opción es realizar la imputación de missing examinando los contaminantes en diferentes zonas geográficas de la Comunidad de Madrid. Según la división establecida, las zonas se definen de la siguiente manera:

* **Zona 1 (interior M30)**: 7 estaciones de tráfico (Escuelas Aguirre, Castellana, Plaza de Castilla, Ramón y Cajal, Cuatro Caminos, Plaza de España y Barrio del Pilar) + 3 estaciones de fondo (Plaza del Carmen, Méndez Álvaro y Retiro).
* **Zona 2 (Sureste)**: 1 estación de tráfico (Moratalaz) y 2 estaciones de fondo (Vallecas y Ensanche de Vallecas).
* **Zona 3 (Noreste)**: 5 estaciones de fondo (Arturo Soria, Sanchinarro, Urbanización Embajada, Barajas Pueblo y Tres Olivos) y 1 estación suburbana (Juan Carlos I).
* **Zona 4 (Noroeste)**: 2 estaciones suburbanas (El Pardo y Casa de Campo).
* **Zona 5 (Suroeste)**: 1 estación de tráfico (Plaza Elíptica) y 2 estaciones de fondo (Farolillo y Villaverde).

De esta forma, realizaremos la imputación de los valores que no han sido validados.

In [ ]:
# Dataframe con todas las zonas
df_zonas = df.pivot_table(index=['MAGNITUD','ANO', 'MES', 'DIA', 'date'], columns='ESTACION', values='VALOR')
df_zonas = df_zonas.reset_index()
df_zonas.drop(['ANO','MES','DIA'],axis=1, inplace=True)
df_zonas = df_zonas.set_index('date')

# Renombrar columnas
cols = df_zonas.columns
nombres = {col: str(col) for col in cols}
df_zonas = df_zonas.rename(columns=nombres)

# Renombrar magnitudes
nombres_magnitudes = {1: 'SO2', 8: 'NO2', 9: 'PM2.5', 10: 'PM10', 14: 'O3'}
df_zonas['MAGNITUD'] = df_zonas['MAGNITUD'].map(nombres_magnitudes)

# Visualizacion
df_zonas.head(5)

In [ ]:
# Comprobar que no se toma el valor 0.0
#suma = 0
#for col in df_zonas:
#    suma = suma + len(df_zonas[df_zonas[col]==0])
#    
#print("Numero de veces que se toma el valor 0:" , suma)
#
# Cambiar valores NaN por 0
#df_zonas = df_zonas.fillna(0)

In [ ]:
# Indices de las estaciones en las diferentes zonas
zona1 = ['8','48','50','11','10','4','5','3','47','9'] # la 3,5,9 y 10 no estan --> buscar como incluirlas
zona2 = ['20','13','54'] # la 20 y 13 no estan
zona3 = ['16','57','55','27','86','59'] # la 86 no esta
zona4 = ['58','24']
zona5 = ['56','18','17']

# Dataframe por zonas
# Zona 1
zona1 = df_zonas[['8','48','50','11','4','47']]

# Zona 2
zona2 = df_zonas[['54']]

# Zona 3
zona3 = df_zonas[['16','57','55','27','59']]

# Zona 4
zona4 = df_zonas[['58', '24']]

# Zona 5
zona5 = df_zonas[['56','18','17']]

Para imputar los valores faltantes, procederemos de la siguiente manera:

1. Si exiten mediciones del contaminante en otras estaciones de la misma zona, es decir, si alguna de las columnas tiene un valor distinto de NaN, imputaremos la media de esos valores.
2. En caso contrario,  utilizaremos la técnica de interpolación lineal.

A continuación, creamos una función para llevar a cabo este proceso:

In [ ]:
def imputar_missing(df):
    # Imputar missing cuando hay datos de otras estaciones
    for i in range(len(df)):
        fila = df.iloc[i,:]
        if (fila == -1).any():
            indices = np.where(df.iloc[i,:] == -1)[0]
            suma = fila.sum() + len(indices)
            if (suma) > 0:
                media = suma/(sum(fila>0))
                fila[indices] = media    
                
    # Imputar missing cuando no hay datos de otras estaciones
    for col in df:
        df[col] = df[col].replace(-1, np.nan)
        df[col] = df[col].interpolate()
    
    return df

Realizamos primero la imputación de los valores faltantes en la **Zona 1 (M30 interior)**.

In [ ]:
# Numero de valores faltantes en las diferentes variables
print("Numero de valores no verificados en las diferentes estaciones de la Zona 1:", )
print(zona1.isin([-1]).sum())

In [ ]:
# Imputacion de missing
zona1 = imputar_missing(zona1)

Verificamos que no haya valores no verficados después de la imputación.

In [ ]:
# Numero de valores faltantes en las diferentes variables
print("Numero de valores no verificados en las diferentes estaciones de la Zona 1:", )
print(zona1.isin([-1]).sum())

Procedemos de manera análoga con el resto de zonas.

In [ ]:
# Imputacion de missing
zona2 = imputar_missing(zona2)
zona3 = imputar_missing(zona3)
zona4 = imputar_missing(zona4)
zona5 = imputar_missing(zona5)

Comprobamos nuevamente que no existan valores nulos después de la imputación en ninguno de los dataframe de las diferentes zonas.

In [ ]:
# Numero de valores faltantes en las diferentes variables 
print("Numero de valores no verificados en las diferentes estaciones de la Zona 2:", )
print(zona2.isin([-1]).sum())
# Numero de valores faltantes en las diferentes variables
print("\nNumero de valores no verificados en las diferentes estaciones de la Zona 3:", )
print(zona3.isin([-1]).sum())
# Numero de valores faltantes en las diferentes variables
print("\nNumero de valores no verificados en las diferentes estaciones de la Zona 4:", )
print(zona4.isin([-1]).sum())
# Numero de valores faltantes en las diferentes variables
print("\nNumero de valores no verificados en las diferentes estaciones de la Zona 5:", )
print(zona5.isin([-1]).sum())

Finalmente, creamos una base de datos que agrupa todas las zonas y bases de datos específicas para cada zona definida.

In [ ]:
# Crear el dataframe con las zonas de estudio
df_estaciones = pd.concat([zona1, zona2, zona3, zona4, zona5], axis=1)
df_estaciones['MAGNITUD'] = df_zonas['MAGNITUD']

In [ ]:
# Magnitudes para las zonas
zona1['MAGNITUD'] = df_estaciones['MAGNITUD']
zona2['MAGNITUD'] = df_estaciones['MAGNITUD']
zona3['MAGNITUD'] = df_estaciones['MAGNITUD']
zona4['MAGNITUD'] = df_estaciones['MAGNITUD']
zona4['MAGNITUD'] = df_estaciones['MAGNITUD']
zona5['MAGNITUD'] = df_estaciones['MAGNITUD']

## 1.3 Análisis descriptivo

Para continuar con el estudio, llevamos a cabo un análisis descriptivo de los datos. Para ello, creamos dos dataframes: uno que contiene la información de las magnitudes o contaminantes y otro que está agrupado por zonas geográficas.

**Por magnitudes**

En primer lugar, analizamos el dataframe relacionado con los contaminantes. 

In [ ]:
# Dataframe con las series temporales de los contaminantes
df_magnitudes = df.pivot_table(index=['ESTACION', 'ANO', 'MES', 'DIA', 'date'], columns='MAGNITUD', values='VALOR')
df_magnitudes = df_magnitudes.reset_index()

# Renombrar columnas
df_magnitudes.columns = ['ESTACION','ANO','MES','DIA','date','SO2', 'NO2', 'PM2.5', 'PM10','O3']

# Visualizacion
df_magnitudes.head(5)

Como se puede observar, no todas las estaciones miden todos los contaminantes. Por tanto, para ajustar un modelo de series temporales, tomamos el promedio diario del contaminante utilizando todas las estaciones de medición.

In [ ]:
# Calculo promedio diario
ts_magnitudes = df_magnitudes.groupby('date').mean()
ts_magnitudes.drop(columns=['ESTACION', 'MES'], inplace=True)

El siguiente gráfico muestra las tendencias medias por año de los diferentes contaminantes. Cada línea representa un contaminante específico. En el eje x se encuentran los años, mientras que en el eje y se muestran los valores promedio.

In [ ]:
# Calcular las medias por año
contaminantes = ['SO2', 'NO2', 'PM2.5', 'PM10', 'O3']
media_anual = ts_magnitudes.groupby('ANO').mean()

# Grafico
plt.figure(figsize=(10, 6))

colors = ['blue', 'green', 'orange', 'red', 'purple']
for i, contaminante in enumerate(contaminantes):
    plt.plot(media_anual.index, media_anual[contaminante], color=colors[i], label=contaminante)

plt.title('Tendencias Medias por Año - Contaminantes')
plt.xlabel('Año')
plt.ylabel('Valor Promedio')
plt.legend() # Buscar mejor sitio
plt.grid(True)

plt.show()

Como se mostró previamente en la figura [Indices parciales](#indices-parciales), podemos crear un índice parcial por contaminante donde se utiliza la siguiente nomenclatura:
* 1: Muy Bueno
* 2: Bueno
* 3: Regular
* 4: Malo
* 5: Muy Malo

In [ ]:
def indice_parcial(valores, magnitud):
    # Etiqueta de calidad del aire
    labels = [1,2,3,4,5]
    
    # Indice segun las diferentes magnitudes
    if magnitud == 'SO2':
        bins = [0,50,100,350,500,float("inf")]
        indice = pd.cut(valores, bins=bins, labels=labels)
        
    elif magnitud == 'NO2':
        bins = [0,50,100,200,400,float("inf")]
        indice = pd.cut(valores, bins=bins, labels=labels)
    
    elif magnitud == 'PM2.5':
        bins = [0,15,30,55,110,float("inf")]
        indice = pd.cut(valores, bins=bins, labels=labels)
    
    elif magnitud == 'PM10':
        bins = [0,25,50,90,180,float("inf")]
        indice = pd.cut(valores, bins=bins, labels=labels)
    
    elif magnitud == 'O3':
        bins = [0,60,120,180,240,float("inf")]
        indice = pd.cut(valores, bins=bins, labels=labels)
        
    else:
        print('El contaminante no es valido')
        
    return indice

Además, podemos crear un índice general  que se obtiene a partir de los índices parciales y se calcula tomando el valor más alto entre ellos. De esta manera, el índice general representa el nivel de restricción más alto entre todos los contaminantes considerados.

In [ ]:
def indice_general(arr):
    indice = np.max(arr)
    return indice

Aplicamos estas funciones a la base de datos agrupada por contaminantes para generar los índices parciales correspondientes a cada contaminante, así como el índice general que engloba todas las mediciones.

In [ ]:
# Indice para S02
ts_magnitudes['I1'] = indice_parcial(ts_magnitudes['SO2'], 'SO2')

# Indice para N02
ts_magnitudes['I2'] = indice_parcial(ts_magnitudes['NO2'], 'NO2')

# Indice para PM2.5
ts_magnitudes['I3'] = indice_parcial(ts_magnitudes['PM2.5'], 'PM2.5')

# Indice para PM10
ts_magnitudes['I4'] = indice_parcial(ts_magnitudes['PM10'], 'PM10')

# Indice para O3
ts_magnitudes['I5'] = indice_parcial(ts_magnitudes['O3'], 'O3')

# Indice general
ts_magnitudes['ICA'] = ts_magnitudes.loc[:,'I1':'I5'].apply(indice_general,axis=1)

# Visualizacion
ts_magnitudes.head(5)

**Por zonas geográficas**

**Nota:** Hacer un dataframe que conste de:

* filas: las diferentes zonas 
* columnas: indices parcilales e indice general para cada zona

Ver como hacer esto en una función.

Durante el proceso de imputación de los valores no verificados, habíamos examinado la distribución por zonas. 

Para calcular el índice parcial de cada contaminante en función de la zona, calculamos la media para cada fila del dataframe y utilizamos ese valor para generar el índice parcial correspondiente.

In [ ]:
contaminantes

In [ ]:
# Zona 1
zona1_media = zona1.groupby(['date','MAGNITUD']).mean()
zona1_media = zona1_media.mean(axis=1).reset_index()
zona1_media = zona1_media.rename(columns={0: 'valor'})
zona1_media = zona1_media.pivot_table(index=['date'], columns='MAGNITUD', values='valor')
zona1_media = zona1_media[contaminantes]
# Indice para S02
zona1_media['I1'] = indice_parcial(zona1_media['SO2'], 'SO2')

# Indice para N02
zona1_media['I2'] = indice_parcial(zona1_media['NO2'], 'NO2')

# Indice para PM2.5
zona1_media['I3'] = indice_parcial(zona1_media['PM2.5'], 'PM2.5')

# Indice para PM10
zona1_media['I4'] = indice_parcial(zona1_media['PM10'], 'PM10')

# Indice para O3
zona1_media['I5'] = indice_parcial(zona1_media['O3'], 'O3')

# Indice general
zona1_media['ICA'] = zona1_media.loc[:,'I1':'I5'].apply(indice_general,axis=1)

# Visualizacion
zona1_media.head(5)

In [ ]:
# Zona 2
zona2_media = zona2.groupby(['date','MAGNITUD']).mean()
zona2_media = zona2_media.mean(axis=1).reset_index()
zona2_media = zona2_media.rename(columns={0: 'valor'})
zona2_media = zona2_media.fillna(0)
zona2_media = zona2_media.pivot_table(index=['date'], columns='MAGNITUD', values='valor')
zona2_media = zona2_media[contaminantes]

# Indice para S02
zona2_media['I1'] = indice_parcial(zona2_media['SO2'], 'SO2')

# Indice para N02
zona2_media['I2'] = indice_parcial(zona2_media['NO2'], 'NO2')

# Indice para PM2.5
zona2_media['I3'] = indice_parcial(zona2_media['PM2.5'], 'PM2.5')

# Indice para PM10
zona2_media['I4'] = indice_parcial(zona2_media['PM10'], 'PM10')

# Indice para O3
zona2_media['I5'] = indice_parcial(zona2_media['O3'], 'O3')

# Indice general
zona2_media['ICA'] = zona2_media.loc[:,'I1':'I5'].apply(indice_general,axis=1)

# Visualizacion
zona2_media.head(5)

In [ ]:
# Zona 3
zona3_media = zona3.groupby(['date','MAGNITUD']).mean()
zona3_media = zona3_media.mean(axis=1).reset_index()
zona3_media = zona3_media.rename(columns={0: 'valor'})
zona3_media = zona3_media.pivot_table(index=['date'], columns='MAGNITUD', values='valor')
zona3_media = zona3_media[contaminantes]
zona3_media

In [ ]:
# Zona 3
zona3_media = zona3.groupby(['date','MAGNITUD']).mean()
zona3_media = zona3_media.mean(axis=1).reset_index()
zona3_media = zona3_media.rename(columns={0: 'valor'})
zona3_media = zona3_media.pivot_table(index=['date'], columns='MAGNITUD', values='valor')
zona3_media = zona3_media[contaminantes]

# Indice para S02
zona3_media['I1'] = indice_parcial(zona3_media['SO2'], 'SO2')

# Indice para N02
zona3_media['I2'] = indice_parcial(zona3_media['NO2'], 'NO2')

# Indice para PM2.5
zona3_media['I3'] = indice_parcial(zona3_media['PM2.5'], 'PM2.5')

# Indice para PM10
zona3_media['I4'] = indice_parcial(zona3_media['PM10'], 'PM10')

# Indice para O3
zona3_media['I5'] = indice_parcial(zona3_media['O3'], 'O3')

# Indice general
zona3_media['ICA'] = zona3_media.loc[:,'I1':'I5'].apply(indice_general,axis=1)

# Visualizacion
zona3_media.head(5)

In [ ]:
# Zona 3
zona4_media = zona4.groupby(['date','MAGNITUD']).mean()
zona4_media = zona4_media.mean(axis=1).reset_index()
zona4_media = zona4_media.rename(columns={0: 'valor'})
zona4_media = zona4_media.pivot_table(index=['date'], columns='MAGNITUD', values='valor')
zona4_media = zona4_media[contaminantes]

# Indice para S02
zona4_media['I1'] = indice_parcial(zona4_media['SO2'], 'SO2')

# Indice para N02
zona4_media['I2'] = indice_parcial(zona4_media['NO2'], 'NO2')

# Indice para PM2.5
zona4_media['I3'] = indice_parcial(zona4_media['PM2.5'], 'PM2.5')

# Indice para PM10
zona4_media['I4'] = indice_parcial(zona4_media['PM10'], 'PM10')

# Indice para O3
zona4_media['I5'] = indice_parcial(zona4_media['O3'], 'O3')

# Indice general
zona4_media['ICA'] = zona4_media.loc[:,'I1':'I5'].apply(indice_general,axis=1)

# Visualizacion
zona4_media.head(5)

In [ ]:
# Zona 5
zona5_media = zona5.groupby(['date','MAGNITUD']).mean()
zona5_media = zona5_media.mean(axis=1).reset_index()
zona5_media = zona5_media.rename(columns={0: 'valor'})
zona5_media = zona5_media.pivot_table(index=['date'], columns='MAGNITUD', values='valor')
zona5_media = zona5_media[contaminantes]

# Indice para S02
zona5_media['I1'] = indice_parcial(zona5_media['SO2'], 'SO2')

# Indice para N02
zona5_media['I2'] = indice_parcial(zona5_media['NO2'], 'NO2')

# Indice para PM2.5
zona5_media['I3'] = indice_parcial(zona5_media['PM2.5'], 'PM2.5')

# Indice para PM10
zona5_media['I4'] = indice_parcial(zona5_media['PM10'], 'PM10')

# Indice para O3
zona5_media['I5'] = indice_parcial(zona5_media['O3'], 'O3')

# Indice general
zona5_media['ICA'] = zona5_media.loc[:,'I1':'I5'].apply(indice_general,axis=1)

# Visualizacion
zona5_media.head(5)

**Nota:** Ver cómo juntar los dataframes en 1 solo que refleje las zonas.

Vamos a crear un mapa de la Comunidad de Madrid con las zonas agrupadas y representar el índice de calidad del aire general en cada zona mediante el uso de diferentes colores.

In [ ]:
#!pip install folium

In [ ]:
# Librerias
import folium
from folium.plugins import MarkerCluster
from folium.vector_layers import Polygon

In [ ]:
# Crear mapa centrado en la Comunidad de Madrid
madrid_map = folium.Map(location=[40.4168, -3.7038], zoom_start=11)

In [ ]:
# Definir las zonas 
# Coordenadas zona 1
interior_coordinates = [[40.419, -3.717], [40.419, -3.686], 
                        [40.434, -3.686], [40.434, -3.717]]

# Coordenadas zona 2
radial1_coordinates = [[40.434, -3.717], [40.434, -3.686],
                       [40.442, -3.686], [40.442, -3.717]]

# Coordenadas zona 3
radial2_coordinates = [[40.434, -3.717], [40.434, -3.748],
                       [40.442, -3.748], [40.442, -3.717]]

# Coordenadas zona 4
radial3_coordinates = [[40.434, -3.717], [40.434, -3.779],
                       [40.442, -3.779], [40.442, -3.717]]

# Coordenadas zona 5
radial4_coordinates = [[40.434, -3.717], [40.434, -3.654], 
                       [40.442, -3.654], [40.442, -3.717]]

In [ ]:
# Capas
# Capa zona 1
central_polygon = folium.vector_layers.Polygon(locations=interior_coordinates, color='blue', fill=True, fill_color='green', fill_opacity=0.3)
central_polygon.add_to(madrid_map)

# Capa zona2
radial1_polygon = folium.vector_layers.Polygon(locations=radial1_coordinates, color='blue', fill=True, fill_color='green', fill_opacity=0.3)
radial1_polygon.add_to(madrid_map)

# Capa zona 3
radial2_polygon = folium.vector_layers.Polygon(locations=radial2_coordinates, color='blue', fill=True, fill_color='green', fill_opacity=0.3)
radial2_polygon.add_to(madrid_map)

# Capa zona 4
radial3_polygon = folium.vector_layers.Polygon(locations=radial3_coordinates, color='blue', fill=True, fill_color='green', fill_opacity=0.3)
radial3_polygon.add_to(madrid_map)

# Capa zona 5
radial4_polygon = folium.vector_layers.Polygon(locations=radial4_coordinates, color='blue', fill=True, fill_color='green', fill_opacity=0.3)
radial4_polygon.add_to(madrid_map)

In [ ]:
# Visualizacion
madrid_map

## 1.4 Outliers

Podemos estudiar los outliers de varias formas. En primer lugar, podemos analizar los outliers por zonas geográficas.

En primer lugar, definimos una función para crear lo gráficos boxplot

In [ ]:
# Recodificacion de las magnitudes con sus respectivos nombres
nombres_magnitudes = {1: 'SO2', 8: 'NO2', 9: 'PM2.5', 10: 'PM10', 14: 'O3'}
magnitudes = ['SO2', 'NO2', 'PM2.5', 'PM10', 'O3']

# Funcion para crear boxplot
def boxplot_zona(df):
    # Crear columna 'MAGNITUD' en el dataframe
    df['MAGNITUD'] = df_estaciones['MAGNITUD']
    df['MAGNITUD'] = df['MAGNITUD'].map(nombres_magnitudes)
    
    # Crear figura y ejes
    fig, axes = plt.subplots(5, 1, figsize=(10, 15))
    
    # Iterar sobre las magnitudes y crear un subplot para cada una
    for i, magnitud in enumerate(magnitudes):
        # Filtrar los datos por magnitud y reemplazar los ceros por NaN
        data = df[df['MAGNITUD'] == magnitud].replace(0, np.nan) 
        
        # Crear el boxplot en el subplot correspondiente
        sns.boxplot(data=data, ax=axes[i])
        axes[i].set_title(f'Boxplot de la Magnitud {magnitud}')
        axes[i].set_xlabel('Magnitud')
        axes[i].set_ylabel('Valores')
        
    # Ajustar espacio entre subplots
    plt.tight_layout()
    
    return plt.show()

Estudiamos los boxplot en las diferentes zonas geográficas.

Comenzamos por la **Zona 1 (M30 interior)**.

In [ ]:
# Zona1
boxplot_zona(zona1)

In [ ]:
# Zona 2
boxplot_zona(zona2)

In [ ]:
# Zona 3
boxplot_zona(zona3)

In [ ]:
# Zona 4
boxplot_zona(zona4)

In [ ]:
# Zona 5
boxplot_zona(zona5)